### 数据预处理
1. 训练数据加载

In [9]:
import pandas as pd
import numpy as np
# 这里的编码类型可以用sublime打开查看左下角
base_train = pd.read_csv('tables/base-train.csv',engine='python',encoding='gb2312')
knowledge_train = pd.read_csv('tables/knowledge-train.csv',engine='python',encoding='utf8')
money_train = pd.read_csv('tables/money-train.csv',engine='python',encoding='utf8')
year_train = pd.read_csv('tables/year-train.csv',engine='python',encoding='gb2312')

2. 处理base_train数据

    2.1 将中文特征编码

    2.2 去除无关信息

    2.3 缺失值处理(平均值)

In [10]:
map_industry = {'零售业':1,'服务业':2,'工业':3,'商业服务业':4,'社区服务':5,'交通运输业':6}
map_type_enterprise = {'有限责任公司':10,'合伙企业':20,'股份有限公司':30,'农民专业合作社':40,'集体所有制企业':50}
map_type_controller = {'自然人':10,'企业法人':20}
# 2.1 给中文编码
base_train['行业'] = base_train['行业'].map(map_industry)
base_train['企业类型'] = base_train['企业类型'].map(map_type_enterprise)
base_train['控制人类型'] = base_train['控制人类型'].map(map_type_controller)
# 2.2 去除无关信息
base_train_data = base_train.drop(columns=['区域'])
# 2.3 缺失值处理（平均值）
for column in list(base_train_data.columns[base_train_data.isnull().sum() > 0]):
    a = base_train_data[column].mean()
    base_train_data[column].fillna(a,inplace=True)

3. 处理knowledge_train数据

    3.1 缺失值处理(01数据类型，round四舍五入取值即可)
4. 处理money_train和year_train数据

    4.1 根据ID和year两个属性合并两个数据集

    4.2 提取15、16和17的数据

    4.3 将15、16和17年的数据合并成一张表

In [11]:
# 3.1 缺失值处理（平均数）
for column in list(knowledge_train.columns[knowledge_train.isnull().sum() > 0]):
    a = round(knowledge_train[column].mean())
    knowledge_train[column].fillna(a,inplace=True)
# 4.1 根据ID和year属性合并两个数据集
money_year_train = pd.merge(money_train,year_train,on=['ID','year'],how='outer')
# 4.2 提取15、16和17年的数据
money_year_train_15 = money_year_train.loc[money_year_train['year']==2015].add_suffix('_15')
money_year_train_15.rename(columns={'ID_15':'ID','year_15':'year'},inplace=True)
money_year_train_16 = money_year_train.loc[money_year_train['year']==2016].add_suffix('_16')
money_year_train_16.rename(columns={'ID_16':'ID','year_16':'year'},inplace=True)
money_year_train_17 = money_year_train.loc[money_year_train['year']==2017].add_suffix('_17')
money_year_train_17.rename(columns={'ID_17':'ID','year_17':'year'},inplace=True)
# 4.3 合并3年的数据
money_year_train_1516=pd.merge(money_year_train_15,money_year_train_16,on='ID',how='outer')
money_year_train_151617=pd.merge(money_year_train_1516,money_year_train_17,on='ID',how='outer')

5. 整合所有的表

    5.1 将base和knowledge表连接起来(采用内连接)

    5.2 将所有表连接

    5.3 去除多余的属性（year_x,year_y,year）

    5.4 填充缺失值（平均值）

In [12]:
# 5.1 将base和knowledge表连接起来(采用外连接)
base_knowledge_train = pd.merge(base_train_data,knowledge_train,on='ID',how='outer')
# 5.2 将所有表连接
train_data = pd.merge(money_year_train_151617,base_knowledge_train,on='ID',how='outer')
# 5.3 去除多余的属性（year_x,year_y,year）
train_data = train_data.drop(columns=['year_x','year_y','year'])
# 5.4 填充缺失值（平均值）
for column in list(train_data.columns[train_data.isnull().sum() > 0]):
    a = int(train_data[column].mean())
    train_data[column].fillna(a,inplace=True)
# 最终训练数据
train_data.to_csv('tables/train.csv')